In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
np.random.seed(1234)
N = 1000

#### data generating

The model:

$$
y_i = 1 + x_{1i} + x_{2i} + \epsilon_i
$$

where $x_{1i}$, $x_{2i}$, $ε_i$ are mutually independent standard normal random variables.

In [4]:
# generating standard normal dist random numbers
x1 = np.random.normal(size=N)
x2 = np.random.normal(size=N)
ε = np.random.normal(size=N)

y = 1 + x1 + x2 + ε

## do OLS regression manually

In [5]:
X = np.empty((N, 3)) # N by 3 matrix
X[:, 0] = 1 # consta_nt term
X[:, 1] = x1 # fill with x1 values
X[:, 2] = x2 # fill with x2 values

In [6]:
β_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
β_hat

array([1.04229289, 1.03666859, 1.05311438])

In [7]:
ϵ_hat = y - X @ β_hat

## (a) t-statistics

$$
\hat{\sigma}^2 = \frac{\sum_{i=1}^{N} \hat{\epsilon_i}^2}{N-K}
$$

where $N=1000$ and $K=3$ in this case.

$$
Var(\hat{\beta} \mid X) = \hat{\sigma} ^ 2 (X^\prime X) ^ {-1}
$$

In [8]:
σ_hat = np.sqrt((ϵ_hat ** 2).sum() / (N - 3))

In [9]:
σ_hat

0.9910191890886281

In [10]:
# auxiliary residuals of regressing X1 on constant and X2
γ_hat = np.linalg.inv(X[:, [0, 2]].T @ X[:, [0, 2]]) @ (X[:, [0, 2]].T @ x1)
ν1_hat = x1 - X[:, [0, 2]] @ γ_hat

In [22]:
denominator = np.sqrt(σ_hat ** 2 / (ν1_hat.T @ ν1_hat))

In [23]:
numerator = β_hat[1] - 0

In [24]:
numerator / denominator 

32.18682240599345

In [14]:
np.sqrt(np.diag(σ_hat ** 2 / (X.T @ X)))

array([0.03133878, 0.032202  , 0.03159306])

## (b) F-statistics

$H_0$:

$$
C \hat{\beta} = 0
$$

where $C=[0 \ I_2]$.

Wald test:

$$
\frac{(C\hat{\beta}-0)^\prime[C(X^\prime X)^{-1}C^\prime]^{-1}(C\hat{\beta}-0)/r}{\hat{U}^\prime\hat{U}/(N-K)} \sim F(r, N-K)
$$

where $r=2$.

In [15]:
denominator = (ϵ_hat @ ϵ_hat) / (N - 3)

In [16]:
C = np.zeros((2, 3))
C[:, 1:] = np.eye(2)
numerator = (C @ β_hat).T @ np.linalg.inv(C @ np.linalg.inv(X.T @ X) @ C.T) @ (C @ β_hat) / 2

In [17]:
F = numerator / denominator
F

1083.4621643692135

# Using Statsmodels

In [18]:
import statsmodels.api as sm

In [19]:
results = sm.OLS(y, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.684
Method:                 Least Squares   F-statistic:                     1083.
Date:                Sun, 03 Feb 2019   Prob (F-statistic):          9.70e-251
Time:                        14:01:42   Log-Likelihood:                -1408.4
No. Observations:                1000   AIC:                             2823.
Df Residuals:                     997   BIC:                             2838.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0423      0.031     33.225      0.0

In [25]:
# t-test
results.t_test([0., 1., 0.]).statistic

array([[32.18682241]])

In [21]:
# F-test
results.f_test(C)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[1083.46216437]]), p=9.701336398738257e-251, df_denom=997, df_num=2>